<a href="https://colab.research.google.com/github/marcosferreiracabral/projeto_pyspark_aviacao/blob/main/avaliacao_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Instalação das dependências:

!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Fazendo download:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

# Descompactando os arquivos:
!tar xf spark-3.3.0-bin-hadoop3.tgz

# instalando a findspark:
!pip install -q findspark

# instalando o pyspark:
!pip install -q pyspark


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 1s (251 kB/s)
Reading package lis

In [ ]:
# Configuração das variáveis de ambiente:

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

# Tornar o pyspark "importável":

import findspark

findspark.init('spark-3.3.0-bin-hadoop3')


In [ ]:
# Iniciar uma sessão local:

import pyspark
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').getOrCreate()


In [ ]:
# Importação das bibliotecas para o notebook:

from pyspark.sql.functions import col, expr


# Airports Dataset:

In [ ]:
# Importação dos dados para o PySpark:
# Download do http para arquivo local:

!wget --quiet --show-progress https://drive.google.com/file/d/1cpygs-tfjUqAYqbBAkt3zcVP1h4J_3W_/view?usp=sharing

# Carregar dados do Airports:

df_airports = spark.read.csv("./airports.csv", inferSchema=True, header=True)

# Ver algumas informações sobre os tipos de dados de cada coluna:

df_airports.printSchema()


view?usp=sharing.3      [ <=>                ]  66.55K  --.-KB/s    in 0.004s  
root
 |-- faa: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- alt: integer (nullable = true)
 |-- tz: integer (nullable = true)
 |-- dst: string (nullable = true)



# Data Discovery:

In [ ]:
# Tipo do dado:

type(df_airports)

pyspark.sql.dataframe.DataFrame

In [ ]:
# Lendo o dataframe e impimindo as colunas e linhas:

df_airports.show()


+---+--------------------+----------------+-----------------+----+---+---+
|faa|                name|             lat|              lon| alt| tz|dst|
+---+--------------------+----------------+-----------------+----+---+---+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|      48.0538086|     -122.8106436| 108| -8|  A|
|0W3|Harford County Ai...

In [ ]:
# Lendo o dataframe e impimindo o cabeçalho:

df_airports.head(10) # 10 primeiras linhas.


[Row(faa='04G', name='Lansdowne Airport', lat=41.1304722, lon=-80.6195833, alt=1044, tz=-5, dst='A'),
 Row(faa='06A', name='Moton Field Municipal Airport', lat=32.4605722, lon=-85.6800278, alt=264, tz=-5, dst='A'),
 Row(faa='06C', name='Schaumburg Regional', lat=41.9893408, lon=-88.1012428, alt=801, tz=-6, dst='A'),
 Row(faa='06N', name='Randall Airport', lat=41.431912, lon=-74.3915611, alt=523, tz=-5, dst='A'),
 Row(faa='09J', name='Jekyll Island Airport', lat=31.0744722, lon=-81.4277778, alt=11, tz=-4, dst='A'),
 Row(faa='0A9', name='Elizabethton Municipal Airport', lat=36.3712222, lon=-82.1734167, alt=1593, tz=-4, dst='A'),
 Row(faa='0G6', name='Williams County Airport', lat=41.4673056, lon=-84.5067778, alt=730, tz=-5, dst='A'),
 Row(faa='0G7', name='Finger Lakes Regional Airport', lat=42.8835647, lon=-76.7812318, alt=492, tz=-5, dst='A'),
 Row(faa='0P2', name='Shoestring Aviation Airfield', lat=39.7948244, lon=-76.6471914, alt=1000, tz=-5, dst='U'),
 Row(faa='0S9', name='Jefferson 

In [ ]:
# Contagem das linhas da coluna faa:

df_airports.select(col('faa')).count()


1397

In [ ]:
# Tipo da coluna faa:

df_airports.select(col('faa')).dtypes


[('faa', 'string')]

In [ ]:
# Retorna os dados em um Array:
# df_airports.select(col('faa')).collect()


In [ ]:
# Retorna um novo DataFramecontendo as linhas distintas neste DataFrame:

df_airports.select(col('faa')).distinct().count()


1396

In [ ]:
# Retorna um novo DataFrame contendo as linhas distintas neste DataFrame:

df_airports.select(col('faa')).explain()


== Physical Plan ==
FileScan csv [faa#2869] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/airports.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<faa:string>




In [ ]:
# Substitui os valores nulos:

df_airports.fillna('').show()


+---+--------------------+----------------+-----------------+----+---+---+
|faa|                name|             lat|              lon| alt| tz|dst|
+---+--------------------+----------------+-----------------+----+---+---+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|      48.0538086|     -122.8106436| 108| -8|  A|
|0W3|Harford County Ai...

In [ ]:
# Retorna um novo DataFrameomitindo linhas com valores nulos:

df_airports.dropna().show()


+---+--------------------+----------------+-----------------+----+---+---+
|faa|                name|             lat|              lon| alt| tz|dst|
+---+--------------------+----------------+-----------------+----+---+---+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|      48.0538086|     -122.8106436| 108| -8|  A|
|0W3|Harford County Ai...

In [ ]:
# Função inclui contagem, média, stddev, min e max. Se nenhuma coluna for fornecida, esta função calcula estatísticas para todas as colunas numéricas ou de string:

df_airports.describe(['faa']).show()


+-------+------------------+
|summary|               faa|
+-------+------------------+
|  count|              1397|
|   mean|             141.0|
| stddev|197.46392075515973|
|    min|               04G|
|    max|               ZYP|
+-------+------------------+



Perguntas<br>
Considere o dataset airports.csv para realizar as seguintes tarefas:

1 - Crie a coluna qa_faa e aponte inconsistências da coluna faa de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
F : Indica que não respeita o formator de 3-5 caracteres alfanuméricos.<br>

In [ ]:
df_airports_qa_faa = df_airports.withColumn('qa_faa',
                                            expr("""CASE 
                                                        WHEN faa = '' THEN 'M'
                                                        WHEN faa = 'RLIKE [a-zA-Z0-9] AND length(faa) < 3 AND length(faa) > 5'  THEN 'F'
                                                        ELSE 'result'
                                                    END""")).show()


+---+--------------------+----------------+-----------------+----+---+---+------+
|faa|                name|             lat|              lon| alt| tz|dst|qa_faa|
+---+--------------------+----------------+-----------------+----+---+---+------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|result|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|result|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|result|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|result|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|result|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|result|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|result|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|result|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|result|
|0S9|Jefferson C

2 - Crie a coluna qa_name e aponte inconsistências da coluna name de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.

In [ ]:
df_airports_qa_name = df_airports.withColumn('qa_name',
                                            expr("""CASE 
                                                        WHEN name = '' THEN 'M'
                                                        ELSE 'result'
                                                    END""")).show()


+---+--------------------+----------------+-----------------+----+---+---+-------+
|faa|                name|             lat|              lon| alt| tz|dst|qa_name|
+---+--------------------+----------------+-----------------+----+---+---+-------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A| result|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A| result|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A| result|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A| result|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A| result|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A| result|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A| result|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A| result|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U| result|
|0S9

3 - Crie a coluna qa_lat e aponte inconsistências da coluna lat de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [-180, 180].<br>
A : Indica que o valor é alfanumérico.

In [ ]:
df_airports_qa_lat = df_airports.withColumn('qa_lat',
                                            expr("""CASE 
                                                        WHEN lat = '' THEN 'M'
                                                        WHEN lat = ' lat < -180 AND lat > 180' THEN 'I'
                                                        WHEN lat = 'RLIKE [a-zA-Z0-9]'  THEN 'A'
                                                        ELSE 'result'
                                                    END""")).show()


+---+--------------------+----------------+-----------------+----+---+---+------+
|faa|                name|             lat|              lon| alt| tz|dst|qa_lat|
+---+--------------------+----------------+-----------------+----+---+---+------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|result|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|result|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|result|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|result|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|result|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|result|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|result|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|result|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|result|
|0S9|Jefferson C

4 - Crie a coluna qa_lon e aponte inconsistências da coluna lon de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [-180, 180].<br>
A : Indica que o valor é alfanumérico.

In [ ]:
df_airports_qa_lon = df_airports.withColumn('qa_lon',
                                            expr("""CASE 
                                                        WHEN lon = '' THEN 'M'
                                                        WHEN lon = ' lon < -180 AND lon > 180' THEN 'I'
                                                        WHEN lon = 'RLIKE [a-zA-Z0-9]'  THEN 'A'
                                                        ELSE 'result'
                                                    END""")).show()


+---+--------------------+----------------+-----------------+----+---+---+------+
|faa|                name|             lat|              lon| alt| tz|dst|qa_lon|
+---+--------------------+----------------+-----------------+----+---+---+------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|result|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|result|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|result|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|result|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|result|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|result|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|result|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|result|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|result|
|0S9|Jefferson C

5 - Crie a coluna qa_alt e aponte inconsistências da coluna alt de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [0,+∞).<br>
A : Indica que o valor é alfanumérico.<br>

In [ ]:
df_airports_qa_lon = df_airports.withColumn('qa_lon',
                                            expr("""CASE 
                                                        WHEN lon = '' THEN 'M'
                                                        WHEN lon = ' lon < 0 AND lon > MAX(lon)' THEN 'I'
                                                        WHEN lon = 'RLIKE [a-zA-Z0-9]'  THEN 'A'
                                                        ELSE 'result'
                                                    END""")).show()


+---+--------------------+----------------+-----------------+----+---+---+------+
|faa|                name|             lat|              lon| alt| tz|dst|qa_lon|
+---+--------------------+----------------+-----------------+----+---+---+------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|result|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|result|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|result|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|result|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|result|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|result|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|result|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|result|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|result|
|0S9|Jefferson C

6 - Crie a coluna qa_tz e aponte inconsistências da coluna tz de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [-11, +14].<br>
A : Indica que o valor é alfanumérico.

In [ ]:
df_airports_qa_tz = df_airports.withColumn('qa_tz',
                                            expr("""CASE 
                                                        WHEN  tz = '' THEN 'M'
                                                        WHEN  tz = '  tz < -11 AND  tz > +14' THEN 'I'
                                                        WHEN  tz = 'RLIKE [a-zA-Z0-9]'  THEN 'A'
                                                        ELSE 'result'
                                                    END""")).show()


+---+--------------------+----------------+-----------------+----+---+---+------+
|faa|                name|             lat|              lon| alt| tz|dst| qa_tz|
+---+--------------------+----------------+-----------------+----+---+---+------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|result|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|result|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|result|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|result|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|result|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|result|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|result|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|result|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|result|
|0S9|Jefferson C

7 - Crie a coluna qa_dst e aponte inconsistências da coluna dst de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
C : Indica que o valor não pertence a nenhuma das categorias esperadas: E, A, S, O, Z, N, U<br>
N : Indica que o valor é numérico.

In [ ]:
# N : Indica que o valor é numérico:

df_airports.select(col('dst')).dtypes

[('dst', 'string')]

In [ ]:
df_airports_qa_dst = df_airports.withColumn('qa_dst',
                                            expr("""CASE
                                                        WHEN dst = ''  THEN 'M'
                                                        WHEN dst = 'E' THEN 'C'
                                                        WHEN dst = 'A' THEN 'C'
                                                        WHEN dst = 'S' THEN 'C'
                                                        WHEN dst = 'O' THEN 'C'
                                                        WHEN dst = 'Z' THEN 'C'
                                                        WHEN dst = 'N' THEN 'C'
                                                        WHEN dst = 'U' THEN 'C'
                                                        WHEN dst = 'RLIKE [a-zA-Z0-9]'  THEN 'A'
                                                        ELSE 'result'
                                            END""")).show()


+---+--------------------+----------------+-----------------+----+---+---+------+
|faa|                name|             lat|              lon| alt| tz|dst|qa_dst|
+---+--------------------+----------------+-----------------+----+---+---+------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|     C|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|     C|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|     C|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|     C|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|     C|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|     C|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|     C|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|     C|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|     C|
|0S9|Jefferson C

# Planes Dataset:

In [ ]:
# Importação dos dados para o PySpark:
# Download do http para arquivo local:

!wget --quiet --show-progress https://drive.google.com/file/d/18Ru1UuLHv9d6nPBWj5-T_ikyJAHhaDI2/view?usp=sharing

# Carregar dados do Planes:

df_planes = spark.read.csv("./planes.csv", inferSchema=True, header=True)

# Ver algumas informações sobre os tipos de dados de cada coluna:

df_planes.printSchema()


view?usp=sharing.4      [ <=>                ]  66.55K  --.-KB/s    in 0.002s  
root
 |-- tailnum: string (nullable = true)
 |-- year: string (nullable = true)
 |-- type: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- engines: integer (nullable = true)
 |-- seats: integer (nullable = true)
 |-- speed: string (nullable = true)
 |-- engine: string (nullable = true)



In [ ]:
df_planes.show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N110UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA

In [ ]:
df_planes.fillna('').show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N110UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA

Perguntas<br>
Considere o dataset planes.csv para realizar as seguintes tarefas:

1 - Crie a coluna qa_tailnum e aponte inconsistências da coluna tailnum de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
S : Indica que não tem exatamente 5 caracteres.<br>
F : Indica que não respeita o formato esperado (ex. N1234Z ou N123AZ).<br>
FN : Indica que não inicia com a letra "N".<br>
FE : Indica que contém caracteres inválidos ("I", "O", ou 0 como primeiro digito).<br>

In [ ]:
df_planes_qa_tailnum = df_planes.withColumn('qa_tailnum',
                                            expr("""CASE 
                                                        WHEN  tailnum = '' THEN 'M'
                                                        WHEN  tailnum = 'length(tailnum) = 5' THEN 'S'
                                                        WHEN  tailnum = 'RLIKE [N1234Z] OR [N123AZ]'  THEN 'A'
                                                        WHEN  tailnum = 'RLIKE [N]'  THEN 'FN'
                                                        WHEN  tailnum = 'RLIKE [I OR O OR 0]'  THEN 'FE'
                                                        ELSE 'result'
                                                    END""")).show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_tailnum|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    result|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    result|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    result|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    result|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    result|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    result|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  18

2 - Crie a coluna qa_year e aponte inconsistências da coluna year de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [1950,+∞).

In [ ]:
df_planes_qa_year = df_planes.withColumn('qa_year',
                                        expr("""CASE 
                                                    WHEN  year = '' THEN 'M'
                                                    WHEN  year = '1950 < MAX(year)' THEN 'I'
                                                    ELSE 'result'
                                                END""")).show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_year|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan| result|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan| result|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan| result|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan| result|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan| result|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan| result|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan| result|


3 - Crie a coluna qa_type e aponte inconsistências da coluna type de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
C : Indica que o valor não pertence a nenhuma categoria esperada:<br>
Fixed wing multi engine<br>
Fixed wing single engine<br>
Rotorcraft

In [ ]:
df_planes_qa_type = df_planes.withColumn('qa_type',
                                            expr("""CASE 
                                                        WHEN  type = '' THEN 'M'
                                                        WHEN  type = 'RLIKE [Fixed wing multi engine]
                                                        OR [Fixed wing single engine] OR [Rotorcraft]'  THEN 'C'
                                                        ELSE 'result'
                                                    END""")).show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_type|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan| result|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan| result|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan| result|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan| result|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan| result|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan| result|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan| result|


4 - Crie a coluna qa_manufacturer e aponte inconsistências da coluna manufacturer de acordo com
as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
C : Indica que o valor não pertence a nenhuma categoria esperada:<br>
AIRBUS<br>
BOEING<br>
BOMBARDIER<br>
CESSNA<br>
EMBRAER<br>
SIKORSKY<br>
CANADAIR<br>
PIPER<br>
MCDONNELL DOUGLAS<br>
CIRRUS<br>
BELL<br>
KILDALL GARY<br>
LAMBERT RICHARD<br>
BARKER JACK<br>
ROBINSON HELICOPTER<br>
GULFSTREAM<br>
MARZ BARRY<br>

In [ ]:
df_planes_qa_manufacturer = df_planes.withColumn('qa_manufacturer',
                                            expr("""CASE 
                                                        WHEN  manufacturer = '' THEN 'M'
                                                        WHEN  manufacturer = 'RLIKE [AIRBUS] OR [BARKER JACK]
                                                        OR [BELL] OR [BOEING] OR [BOMBARDIER] OR [CANADAIR]
                                                        OR [CESSNA] OR [CIRRUS] OR [EMBRAER] OR [GULFSTREAM]
                                                        OR [KILDALL GARY] OR [LAMBERT RICHARD] OR [MARZ BARRY]
                                                        OR [MCDONNELL DOUGLAS] OR [PIPER] OR [ROBINSON HELICOPTER]
                                                        OR [SIKORSKY]'  THEN 'C'
                                                        ELSE 'result'
                                                    END""")).show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+---------------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_manufacturer|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+---------------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|         result|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|         result|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|         result|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|         result|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|         result|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|         result|
| N109UW|1999|Fixed wing mul

5 - Crie a coluna qa_model e aponte inconsistências da coluna model de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
F : Indica que não respeita o formato esperado<br>
Modelos AIRBUS devem começar com "A"<br>
Modelos BOEING devem começar com "7"<br>
Modelos BOMBARDIER e CANADAIR devem começar com "CL"<br>
Modelos MCDONNELL DOUGLAS devem começar com "MD" ou "DC"

In [ ]:
df_planes_qa_model = df_planes.withColumn('qa_model',
                                            expr("""CASE 
                                                        WHEN  model = '' THEN 'M'
                                                        WHEN  model = 'RLIKE [AIRBUS] = "A" OR [BOEING] = 7
                                                        OR [BOMBARDIER AND CANADAIR] = "CL"
                                                        OR [MCDONNELL DOUGLAS] "MD" OR "DC"'  THEN 'F'
                                                        ELSE 'result'
                                                    END""")).show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_model|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|

6 - Crie a coluna qa_engines e aponte inconsistências da coluna engines de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [1, 4].<br>
A : Indica que o valor é alfanumérico.

In [ ]:
df_planes_qa_engines = df_planes.withColumn('qa_engines',
                                        expr("""CASE 
                                                    WHEN  engines = '' THEN 'M'
                                                    WHEN  engines = '1 < engines AND engines > 4' THEN 'I'
                                                    WHEN  engines = 'RLIKE [a-zA-Z0-9]'  THEN 'A'
                                                    ELSE 'result'
                                                END""")).show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_engines|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    result|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    result|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    result|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    result|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    result|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    result|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  18

7 - Crie a coluna qa_seats e aponte inconsistências da coluna seats de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br><br>
I : Indica que o valor excede o intervalo .<br>
A : Indica que o valor é alfanumérico.

In [ ]:
df_planes_qa_seats = df_planes.withColumn('qa_seats',
                                        expr("""CASE 
                                                    WHEN  seats = '' THEN 'M'
                                                    WHEN  seats = 'RLIKE [a-zA-Z0-9]'  THEN 'A'
                                                    ELSE 'result'
                                                END""")).show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_seats|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|

8 - Crie a coluna qa_speed e aponte inconsistências da coluna speed de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [50, 150].<br>
A : Indica que o valor é alfanumérico.<br>

In [ ]:
df_planes_qa_speed = df_planes.withColumn('qa_speed',
                                        expr("""CASE 
                                                    WHEN  speed = '' THEN 'M'
                                                    WHEN  speed = '50 < speed AND speed > 150' THEN 'I'
                                                    WHEN  speed = 'RLIKE [a-zA-Z0-9]'  THEN 'A'
                                                    ELSE 'result'
                                                END""")).show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_speed|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|  result|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|

9 - Crie a coluna qa_engine e aponte inconsistências da coluna engine de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
C : Indica que o valor não pertence a nenhuma categoria esperada:<br>
Turbo-fan<br>
Turbo-jet<br>
Turbo-prop<br>
Turbo-shaft<br>
4 Cycle

In [ ]:
df_planes_qa_engine = df_planes.withColumn('qa_engine',
                                            expr("""CASE 
                                                        WHEN  engine = '' THEN 'M'
                                                        WHEN  engine = 'RLIKE [Turbo-fan] OR [Turbo-jet]
                                                        OR [Turbo-prop] OR [Turbo-shaft] OR [4 Cycle]' THEN 'C'
                                                        ELSE 'result'
                                                    END""")).show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|   result|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|   result|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|   result|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|   result|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|   result|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|   result|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|T

# Flights Dataset:

In [ ]:
# Importação dos dados para o PySpark:
# Download do http para arquivo local:

!wget --quiet --show-progress https://drive.google.com/file/d/1ggZ_dBMvAkILRh-YrozqsvsXc2XYUYeG/view?usp=sharing

# Carregar dados do Flights:

df_flights = spark.read.csv("./flights.csv", inferSchema=True, header=True)

# Ver algumas informações sobre os tipos de dados de cada coluna:

df_flights.printSchema()


view?usp=sharing.5      [ <=>                ]  66.55K  --.-KB/s    in 0.002s  
root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)



In [ ]:
df_flights.show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [ ]:
df_flights.count()

10000

In [ ]:
df_flights.fillna('').show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [ ]:
df_flights.fillna('').show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

Perguntas<br>
Considere o dataset flights.csv para realizar as seguintes tarefas:

1 - Crie a coluna qa_year_month_day e aponte inconsistências das colunas year , month , day de
acordo com as regras abaixo.<br>
MY : Indica que está com dado faltante no ano.<br>
MM : Indica que está com dado faltante no mes.<br>
MD : Indica que está com dado faltante no dia.<br>
IY : Indica que o valor excede o intervalo [1950,+∞] no ano.<br>
IM : Indica que o valor excede o intervalo [1, 12] no mês.<br>
ID : Indica que o valor excede o intervalo [1, 31] no dia. No mês de Fevereiro o intervalo é [1, 29].

In [ ]:
df_flights_qa_year_month_day = df_flights.withColumn('qa_year_month_day',
                                        expr("""CASE 
                                                    WHEN  year = '' THEN 'MY'
                                                    WHEN  month = '' THEN 'MM'
                                                    WHEN  day = '' THEN 'MD'
                                                    WHEN  year = '1950 < year AND year > MAX(year)' THEN 'IY'
                                                    WHEN  year = '1 < year AND year > 12' THEN 'IM'
                                                    WHEN  month = '1 < month AND month > 31
                                                    AND RLIKE[month = 2] 1 < month AND month > 29' THEN 'ID'
                                                    ELSE 'result'
                                                END""")).show()
                                                

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_year_month_day|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|           result|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|           result|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|           result|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|           result|
|2014|    3|  9|     754|       -1

2 - Crie a coluna qa_hour_minute e aponte inconsistencias das colunas hour e minute de acordo
com as regras abaixo.<br>
MH : Indica que está com dado faltante na hora.<br>
MM : Indica que está com dado faltante no minuto.<br>
IH : Indica que o valor excede o intervalo na hora [0, 24].<br>
IM : Indica que o valor excede o intervalo no minuto [0, 59].<br>

In [ ]:
df_flights_qa_hour_minute = df_flights.withColumn('qa_hour_minute',
                                        expr("""CASE 
                                                    WHEN  hour = '' THEN 'MH'
                                                    WHEN  hour = '' THEN 'MM'
                                                    WHEN  hour = '0 < hour AND hour > 24' THEN 'IH'
                                                    WHEN  hour = '0 < hour AND hour > 59' THEN 'IM'                                                  
                                                    ELSE 'result'
                                                END""")).show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_hour_minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|        result|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|        result|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|        result|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|        result|
|2014|    3|  9|     754|       -1|    1015|        1| 

3 - Crie a coluna qa_dep_arr_time e aponte inconsistências da coluna dep_time e arr_time de
acordo com as regras abaixo.<br>
MD : Indica que está com dado faltante no dep_time .<br>
MA : Indica que está com dado faltante no arr_time .<br>
FD : Indica que não respeita o formato esperado (HHMM ou HMM) no dep_time .<br>
FA : Indica que não respeita o formato esperado (HHMM ou HMM) no arr_time .<br>

In [ ]:
df_flights_qa_dep_arr_time = df_flights.withColumn('qa_dep_arr_time',
                                        expr("""CASE 
                                                    WHEN  dep_time = '' THEN 'MD'
                                                    WHEN  arr_time = '' THEN 'MA'
                                                    WHEN  dep_time = 'RLIKE [HHMM] OR [HMM]' THEN 'FD'
                                                    WHEN  arr_time = 'LIKE [HHMM] OR [HMM]' THEN 'FA'                                                  
                                                    ELSE 'result'
                                                END""")).show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_dep_arr_time|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|         result|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|         result|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|         result|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|         result|
|2014|    3|  9|     754|       -1|    1015|    

4 - Crie a coluna qa_dep_arr_delay e aponte inconsistências da coluna dep_delay e arr_delay de
acordo com as regras abaixo.<br>
MD : Indica que está com dado faltante no dep_delay .<br>
MA : Indica que está com dado faltante no arr_delay .<br>

In [ ]:
df_flights_qa_dep_arr_delay = df_flights.withColumn('qa_dep_arr_delay',
                                        expr("""CASE 
                                                    WHEN  dep_delay = '' THEN 'MD'
                                                    WHEN  arr_delay = '' THEN 'MA'
                                                    ELSE 'result'
                                                END""")).show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_dep_arr_delay|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|          result|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|          result|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|          result|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|          result|
|2014|    3|  9|     754|       -1|    10

5 - Crie a coluna qa_carrier e aponte inconsistências da coluna carrier de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
F : Indica que não respeita o formato esperado (2 caracteres alfanuméricos).<br>

In [ ]:
df_flights_qa_carrier = df_flights.withColumn('qa_dep_arr_delay',
                                        expr("""CASE 
 						    WHEN  carrier = '' THEN 'M'
                                                    WHEN  carrier = 'RLIKE [a-zA-Z0-9] AND length(carrier) = 2'  THEN 'F'
                                                    ELSE 'result'
                                                END""")).show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_dep_arr_delay|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|          result|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|          result|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|          result|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|          result|
|2014|    3|  9|     754|       -1|    10

6 - Crie a coluna qa_tailnum e aponte inconsistências da coluna tailnum de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
S : Indica que não tem o número de caracteres esperado.<br>
F : Indica que não respeita o formato esperado (ex. N1234Z ou N123AZ).<br>
FN : Indica que não inicia com a letra "N".<br>
FE : Indica que contém caracteres inválidos ("I", "O", ou 0 como primeiro digito).<br>

In [ ]:
df_flights_qa_tailnum = df_flights.withColumn('qa_tailnum',
                                            expr("""CASE 
                                                        WHEN  tailnum = '' THEN 'M'
                                                        WHEN  tailnum = 'length(tailnum) = 5' THEN 'S'
                                                        WHEN  tailnum = 'RLIKE [N1234Z] OR [N123AZ]'  THEN 'A'
                                                        WHEN  tailnum = 'RLIKE [N]'  THEN 'FN'
                                                        WHEN  tailnum = 'RLIKE [I OR O OR 0]'  THEN 'FE'
                                                        ELSE 'result'
                                                    END""")).show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_tailnum|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|    result|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|    result|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|    result|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|    result|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA

7 - Crie a coluna qa_flight e aponte inconsistências da coluna flight de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
F : Indica que não respeita o formato esperado (4 caracteres numéricos).<br>

In [ ]:
df_flights_qa_flight = df_flights.withColumn('qa_dep_arr_delay',
                                        expr("""CASE 
 						                                        WHEN  flight = '' THEN 'M'
                                                    WHEN  flight = 'RLIKE [a-zA-Z0-9] AND length(carrier) = 4'  THEN 'F'
                                                    ELSE 'result'
                                                END""")).show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_dep_arr_delay|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|          result|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|          result|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|          result|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|          result|
|2014|    3|  9|     754|       -1|    10

8 - Crie a coluna qa_origin_dest e aponte inconsistências da coluna origin , dest de acordo com
as regras abaixo.<br>
MO : Indica que está com dado faltante no origin .<br>
MD : Indica que está com dado faltante no dest .<br>
FO : Indica que não respeita o formato esperado (3 caracteres alfanuméricos) no origin .<br>
FD : Indica que não respeita o formato esperado (3 caracteres alfanuméricos) no dest .<br>

In [ ]:
df_flights_qa_origin_dest = df_flights.withColumn('qa_origin_dest',
                                            expr("""CASE 
                                                        WHEN  origin = '' THEN 'MO'
                                                        WHEN  dest = '' THEN 'MD'
                                                        WHEN  origin = 'RLIKE [a-zA-Z0-9] AND length(origin) = 3'  THEN 'FO'
                                                        WHEN  dest = 'RLIKE [a-zA-Z0-9] AND length(dest) = 3'  THEN 'FD'
                                                        ELSE 'result'
                                                    END""")).show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_origin_dest|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|        result|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|        result|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|        result|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|        result|
|2014|    3|  9|     754|       -1|    1015|        1| 

9 - Crie a coluna qa_air_time e aponte inconsistencias da coluna air_time de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [20, 500].<br>

In [ ]:
df_flights_qa_air_time = df_flights.withColumn('qa_hour_minute',
                                        expr("""CASE 
                                                    WHEN  air_time = '' THEN 'M'
                                                    WHEN  air_time = '20 < air_time AND air_time > 500' THEN 'I'                                       
                                                    ELSE 'result'
                                                END""")).show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_hour_minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|        result|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|        result|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|        result|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|        result|
|2014|    3|  9|     754|       -1|    1015|        1| 

10 - Crie a coluna qa_distance e aponte inconsistências da coluna distance de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [50, 3000].<br>

In [ ]:
df_flights_qa_distance = df_flights.withColumn('qa_distance',
                                        expr("""CASE 
                                                    WHEN  distance = '' THEN 'M'
                                                    WHEN  distance = '50 < distance AND distance > 3000' THEN 'I'                                       
                                                    ELSE 'result'
                                                END""")).show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_distance|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|     result|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|     result|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|     result|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|     result|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522

11 - Crie a coluna qa_distance_airtime e aponte inconsistências entre as colunas distance e
air_time de acordo com as regras abaixo.<br>
M : Indica que está com distance ou air_time faltante.<br>
TL : Indica que a viagem é longa de acordo com a condição:<br>
air_time >= distance × 0.1 + 30.<br>
TS : Indica que a viagem é curta de acordo com a condição:<br>
air_time <= distance × 0.1 + 10.<br>
TR : Indica que a viagem é normal caso as duas anteriores não sejam verdade.<br>

In [ ]:
df_flights_qa_distance_airtime = df_flights.withColumn('qa_distance_airtime',
                                        expr("""CASE 
                                                    WHEN  distance = '' THEN 'M'
                                                    WHEN  air_time = 'air_time >= distance × 0.1 + 30' THEN 'TL'
                                                    WHEN  air_time = 'air_time <= distance × 0.1 + 10' THEN 'TS'
                                                    WHEN  air_time = 'TL AND TS = TRUE' THEN 'TR'  
                                                    ELSE 'result'
                                                END""")).show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_distance_airtime|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|             result|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|             result|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|             result|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|             result|
|2014|    3|  9|    